In [6]:
from jupyterscad import view
from  solid2 import * 

In [7]:
# dimensions 
debug_matrix = [0,0,20]
null_matrix = [0,0,0]
debug = True
set_global_fn(200)

base_r = 58
base_d = 20
base_w = 2
gap = 3
tolerance = 0.5

tolerance_matrix = [0,0,-tolerance]

inner_r = base_r 
inner_d = base_d
inner_w = base_w 

outer_r = base_r + base_w + gap
outer_d = base_d
outer_w = base_w

mag_count = 16
mag_x,mag_y,mag_z = [2,6,15]

In [8]:
magnets = []
for magnet in range(0, mag_count):
    magnets += cube([mag_x,mag_y,mag_z]).translate([base_r,0,(base_d-mag_z)/2]).rotate([0,0,magnet*(360/mag_count)])

In [9]:
inner_base = cylinder(inner_d,inner_r,inner_r) 
inner_diff = cylinder(inner_d + 2*tolerance,inner_r - inner_w,inner_r - inner_w ).translate(tolerance_matrix)

inner = inner_base - inner_diff

In [10]:
outer_base = cylinder(outer_d,outer_r+gap,outer_r+gap) 
outer_diff = cylinder(outer_d + 2*tolerance,outer_r+gap - outer_w,outer_r+gap - outer_w ).translate(tolerance_matrix)

outer = outer_base - outer_diff

In [ ]:
inner = inner.color([0,0,0.7])
outer = outer.color([0,0,0.9])
magnets = magnets.color([0.9,0,0])
to_render = [inner, outer, magnets]
to_debug = []

obj = [] 

for rendered in to_render:
    obj += rendered
    
for debugged in to_debug:
    obj += debugged.translate(debug_matrix)
    

In [ ]:
obj.save_as_scad("body.scad")

In [15]:
view(obj)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.7, position=(3.0, 5.0,…